# Agentic Rag and External Memory

## Setup

In [ ]:
from letta_client import Letta

client = Letta(base_url="http://localhost:8283")

In [ ]:
def print_message(message):
    if message.message_type == "reasoning_message":
        print("🧠 Reasoning: " + message.reasoning)
    elif message.message_type == "assistant_message":
        print("🤖 Agent: " + message.content)
    elif message.message_type == "tool_call_message":
        print("🔧 Tool Call: " + message.tool_call.name + "\n" + message.tool_call.arguments)
    elif message.message_type == "tool_return_message":
        print("🔧 Tool Return: " + message.tool_return)
    elif message.message_type == "user_message":
        print("👤 User Message: " + message.content)

## Section 1: Data Sources

### Creating a source

In [ ]:
source = client.sources.create(
    name="employee_handbook",
    embedding="openai/text-embedding-3-small"
)
source

Source(id='source-5eb6b6aa-e23a-4687-8354-919488dfd8e4', name='employee_handbook', description=None, embedding_config=EmbeddingConfig(embedding_endpoint_type='openai', embedding_endpoint='http://jupyter-api-proxy.internal.dlai/rev-proxy/letta', embedding_model='text-embedding-3-small', embedding_dim=2000, embedding_chunk_size=300, handle='openai/text-embedding-3-small', azure_endpoint=None, azure_version=None, azure_deployment=None), metadata=None, created_by_id='user-00000000-0000-4000-8000-000000000000', last_updated_by_id='user-00000000-0000-4000-8000-000000000000', created_at=datetime.datetime(2025, 9, 6, 12, 56, 27), updated_at=datetime.datetime(2025, 9, 6, 12, 56, 27), organization_id='org-00000000-0000-4000-8000-000000000000')

### Uploading a source

In [ ]:
job = client.sources.files.upload(
    source_id=source.id,
    file=open("handbook.pdf", "rb")
)

In [ ]:
job.status

'created'

### Viewing job status over time

In [ ]:
import time
from letta_client import JobStatus

while job.status != 'completed':
    job = client.jobs.retrieve(job.id)
    print(job.status)
    time.sleep(1)

completed


### Viewing job metadata

In [ ]:
job.metadata

{'type': 'embedding',
 'filename': 'handbook.pdf',
 'source_id': 'source-5eb6b6aa-e23a-4687-8354-919488dfd8e4',
 'num_passages': 11,
 'num_documents': 1}

In [ ]:
passages = client.sources.passages.list(
    source_id=source.id,
)
len(passages)

11

### Creating an agent and attaching sources

In [ ]:
agent_state = client.agents.create(
    memory_blocks=[
        {
          "label": "human",
          "value": "My name is Sarah"
        },
        {
          "label": "persona",
          "value": "You are a helpful assistant"
        }
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-3-small"
)

In [ ]:
agent_state = client.agents.sources.attach(
    agent_id=agent_state.id,
    source_id=source.id
)

### Viewing agent's attached sources

In [ ]:
client.agents.sources.list(agent_id=agent_state.id)

[Source(id='source-5eb6b6aa-e23a-4687-8354-919488dfd8e4', name='employee_handbook', description=None, embedding_config=EmbeddingConfig(embedding_endpoint_type='openai', embedding_endpoint='http://jupyter-api-proxy.internal.dlai/rev-proxy/letta', embedding_model='text-embedding-3-small', embedding_dim=2000, embedding_chunk_size=300, handle='openai/text-embedding-3-small', azure_endpoint=None, azure_version=None, azure_deployment=None), metadata=None, created_by_id='user-00000000-0000-4000-8000-000000000000', last_updated_by_id='user-00000000-0000-4000-8000-000000000000', created_at=datetime.datetime(2025, 9, 6, 12, 56, 27), updated_at=datetime.datetime(2025, 9, 6, 12, 56, 27), organization_id='org-00000000-0000-4000-8000-000000000000')]

In [ ]:
passages = client.agents.passages.list(agent_id=agent_state.id)
len(passages)

11

### Messaging agents and referencing attached sources

In [ ]:
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "Search archival for our company's vacation policies"
        }
    ]
)
for message in response.messages:
    print_message(message)

🧠 Reasoning: Searching for any relevant details about vacation policies based on the user's request.
🔧 Tool Call: archival_memory_search
{
  "query": "vacation policies",
  "page": 0,
  "start": 0,
  "request_heartbeat": true
}
🔧 Tool Return: ([{'timestamp': '2025-09-06 12:56:37.482848', 'content': "priorities. Assuch, vacationsarepermittedonlyunderthefollowingcondition: youmust provideanAI agent that matchesorsurpassesyourowncompetenciestofullyperformyourdutiesduringyourabsence.\nTheAI replacement must beequivalentlycompetent inall aspectsof yourrole, ensuringseamlesscontinuityof operations. Youarerequiredtosubmit theAI agent toyourimmediatesupervisorat least fourweekspriortoyourintendedleavedate. Thistimeframeallowsforrigoroustestingandevaluationof theAI'scapabilitiesandreliability.\nTheAI will undergocomprehensiveassessmentstoverifyitsproficiencyandeffectivenessinhandlingyourresponsibilities. Approval of theAI agent isat thesolediscretionof uppermanagement, andsubmissiondoesnot guar

## Section 2: Connecting Data with Custom Tools

### Creating a custom tool

In [ ]:
def query_birthday_db(name: str):
    """
    This tool queries an external database to
    lookup the birthday of someone given their name.

    Args:
        name (str): The name to look up

    Returns:
        birthday (str): The birthday in mm-dd-yyyy format

    """
    my_fake_data = {
        "bob": "03-06-1997",
        "sarah": "07-06-1993"
    }
    name = name.lower()
    if name not in my_fake_data:
        return None
    else:
        return my_fake_data[name]

In [ ]:
birthday_tool = client.tools.upsert_from_function(func=query_birthday_db)

### Creating an agent with access to tools

In [ ]:
agent_state = client.agents.create(
    memory_blocks=[
        {
          "label": "human",
          "value": "My name is Sarah"
        },
        {
          "label": "persona",
          "value": "You are a agent with access to a birthday_db " \
            + "that you use to lookup information about users' birthdays."
        }
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-3-small",
    tool_ids=[birthday_tool.id],
    #tool_exec_environment_variables={"DB_KEY": "my_key"}
)

In [ ]:
# send a message to the agent
response = client.agents.messages.create_stream(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "whens my bday????"
        }
    ]
)
for message in response:
    print_message(message)

🧠 Reasoning: User is asking about their birthday. Looking it up in the birthday database.
🔧 Tool Call: query_birthday_db
{
  "name": "Sarah",
  "request_heartbeat": true
}
🔧 Tool Return: 07-06-1993
🧠 Reasoning: Birthday retrieved successfully. Sending response to user.
🤖 Agent: Your birthday is on July 6, 1993! 🎉
